# **Task 3 - KPIs & Exploratory Data Analysis**

Define 3-6 Key Performance Indicators (KPIs) in addition to those used in the previous tasks and perform exploratory data analysis on multiple sheets in both Excel files. Make sure to document what you are investigating using a combination of markdown and code in the Jupyter notebook. If you are using Python scripts, write markdown alongside the code. Also, document your findings and save all visualizations.

**1. For Vaccination Data:**

1.1 **Vaccination Coverage Rate Municipalities:** The percentage of the population in municipalities that has received at least one dose of the COVID-19 vaccine.

1.2 **Vaccination timeseries** Time span for issuing vaccine doses one and two at county level in 2022.

**2. For COVID-19 Cases Data:**

2.1 **Incidence Rate:** The number of new COVID-19 cases reported in a specific period per 100,000 population.

2.2 **Hospitalization Rate:** The percentage of COVID-19 cases that required intensive care. 

2.3 **Case Fatality Rate (CFR):** The percentage of confirmed COVID-19 cases that resulted in death.

### Import of libraries needed for exploration

In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px

### **Vaccination Data:**

#### 1. **Vaccination Coverage Rate Municipalities:** The percentage of the population in municipalities that has received at least one dose of the COVID-19 vaccine. 

We start with uploading the sheet with data about vaccinated split on municipalities. Then we sort the dataframe in descending order based on the share of the total population that received dose one.

In [2]:
# Vaccinated with at least one and at least two doses per municipality
vaccinated_municipality = pd.read_excel("Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinerade kommun")
vaccinated_municipality.sort_values(by='Andel_dos1', ascending=False)

,KnKod,KnNamn,Antal_dos1,Antal_dos2,Andel_dos1,Andel_dos2
187,1761,Hammarö,13046,12916,0.931591,0.922308
273,2480,Umeå,105536,103798,0.928744,0.913449
193,1780,Karlstad,77359,76392,0.922324,0.910795
103,1262,Lomma,19247,18986,0.921218,0.908725
54,584,Vadstena,6204,6160,0.919929,0.913405
...,...,...,...,...,...,...
101,1260,Bjuv,10379,10106,0.764511,0.744402
97,1231,Burlöv,12720,12388,0.763231,0.743310
114,1277,Åstorp,10562,10269,0.762985,0.741819
17,181,Södertälje,62933,59877,0.716744,0.681939


The code above shows the share of the citizens that received dose one split on municipalities and sorted on 'Andel_dos1' (Share dose one). The first five are the ones with the highest share of vaccinated with dose one and last five rows are the ones with the lowest share of vaccinated with dose one.

From what we can see Botkyrka has the lowest numbers of citizens that received dose one with just 70% of the total population in the municipality. On the other hand we have Hammarö that vaccinated 93% of their population with at lest one dose.

In the graph below we can se all municipalities shares of vaccination one in the country. The value for the x-axis is the percent in the municipality that got 1 dose and the value for y is the municipalities ID-number to group the municipalitites by location in the country.

In [3]:
percent_share_dose_one = vaccinated_municipality['Andel_dos1'] * 100

# Create a new dataframe with renamed columns for the plot
data = pd.DataFrame({
    'Share 1 dose %': percent_share_dose_one,
    'Municipality': vaccinated_municipality['KnNamn'],
    'Municipality ID number': vaccinated_municipality['KnKod']
})
fig = px.strip(
    data, 
    x='Share 1 dose %', 
    y='Municipality ID number',
    hover_name='Municipality', 
    title='Share per Municipality that Received Vaccine 1 Dose',
    color='Municipality',
    template='ggplot2',
)
fig.update_xaxes(title_text='Percentage of Population with 1 Dose')
fig.write_html("Visualizations/share_1_dose_municipalities.html")
fig.show()

In [4]:
vaccinated_gender = pd.read_excel("Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinerade kön")
vaccinated_gender.head(6)

,Kön,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
0,Totalt,7810380,0.858964,Minst 1 dos
1,Totalt,7627588,0.838861,Minst 2 doser
2,Män,3858688,0.845743,Minst 1 dos
3,Män,3759898,0.824090,Minst 2 doser
4,Kvinnor,3951692,0.872279,Minst 1 dos
5,Kvinnor,3867690,0.853737,Minst 2 doser


If we then combine the above knowledge with looking att the total number of vaccinated in the country by gender we can assume that men in Botkyrka municipality are the group that has the lowest rate of vaccinations. 

#### 1.2 **Vaccination timeseries** Time span for issuing vaccine doses one and two at county level in 2022.

To answer this question we take a look at our sheet with data for the timeseries in witch the different counties vaccinated their population. To exemplify it we use a animation frame that show the share of the population vaccinated week by week. It will show dose one as well as dose two.

In [5]:
vaccinated_timeseries = pd.read_excel("Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinerade tidsserie")
fig = px.scatter(
    data_frame=vaccinated_timeseries, 
    x='Andel vaccinerade',
    y='Region',
    color='Region', 
    hover_name='Region', 
    animation_frame='Vecka',
    animation_group='Vaccinationsstatus',
    size='Andel vaccinerade',
    height=800)

fig.write_html("Visualizations/vaccination_timeseries.html")
fig.show()

As seen above all of the counties moves in about the same pase over the year, no larger outliers.

### **COVID-19 Cases Data:**

#### 2.1. **Incidence Rate:** The number of new COVID-19 cases reported in a specific period per 100,000 population.

A hypothesis is that we find a higher number of cases in the counties with the larger cities such as Stockholm, Västra Götaland (Gothenburg) and Skåne (Malmö). This because the interaction between people a higher in the bigger cities compared to the countryside.

In [6]:
weekly_data_county = pd.read_excel("Data/Folkhalsomyndigheten_Covid19.xlsx", sheet_name="Veckodata Region")

fig = px.strip(
    data_frame=weekly_data_county,
    y='Kum_fall_100000inv', 
    x='Region', 
    hover_name='Region',
    hover_data=['år','veckonummer'], 
    color="år", 
    title='Cumulative Number of Cases per 100,000 Citizens 2020-2022',
    labels={
        'Kum_fall_100000inv': 'Cumulative number of cases',
        'Region': 'County',
        'år': 'Year',
        'veckonummer': 'Week'
    }
)
fig.update_xaxes(categoryorder='total ascending')
fig.write_html("Visualizations/cumulative_number_of_cases.html")
fig.show()

From the code above we can see how many people that got infected by covid-19 per week and year split on counties. For exmple we learn here that the county with least number of cases were in Blekinge and Gotland and the highest number of cases where in Örebro and Halland. This would maybe give us a hint that Blekinge and Gotland would have the lowest mortality rate as well, we'll take a look at this later on. In Halland and Örebro we find a higher number of cases then the rest of the country. This result contradicts our hypothesis that the higher values would be found in the 'big city' counties. 

Why this result of number of cases is hard to tell. One reason could be that the two counties in top did more test on their citizens then the rest of the counties. During this period the testing was very different in different counties, especially in the larger regions were the capacity to take test on their citizens were limited. Another reason possible is of course that the number of cases actually were higher as what the data tells us.

#### 2.2 **Hospitalization Rate:** The percentage of COVID-19 cases that required intensive care.

To answer the above question we first group the data by 'region' (county), then we take the max value in the column with cumulative number of cases for each county, and the same for number of cases required hospitalization. If we then divide number of hospitalized cases with total number of cases per region we'll get an answer on which county had the highest percentage of hospitalization cases.

In our previous question we examined what counties had the highest number of cases and here we instead examine what counties has the highest number of severe cases. One could guess this also is interrelated with the vaccination rate in the counties. So we'll start examine the vaccination rate (similar to what we did in exercise 2 but with only dose 1 this time). Here we group the values for dose one in our data by county and calculate the mean values (hence they are split on age groups). 

In [7]:
vaccinated_municipallity_and_age = pd.read_excel("Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name="Vaccinerade kommun och ålder")
df = pd.DataFrame(vaccinated_municipallity_and_age)

county_1_dose_means = df.groupby('Län_namn')['Andel minst 1 dos'].mean()

data = {
    'Counties': county_1_dose_means.index,
    'Mean % 1 Dose': county_1_dose_means.values * 100,
}

fig = px.bar(
    data, 
    x='Counties', 
    y='Mean % 1 Dose',
    title='Mean Vaccination percent for 1 dose by County',
    barmode='group',
    color='Mean % 1 Dose'
)
fig.update_xaxes(tickangle=45, categoryorder='total ascending')
fig.write_html("Visualizations/vaccination_1_dose_county.html")
fig.show()


Above we see that the vaccination rate for 1 dose are quite similar in each county, but in top we find Gotland (who also had the lowest number of cases) and in the bottom we'll find Kronoberg. Kronoberg was in our previous question in the middle of number of cases per 100000 citizens.

In [8]:
total_number_of_cases_county = weekly_data_county.groupby('Region')['Kum_antal_fall'].max()
total_number_of_hospitalized_county = weekly_data_county.groupby('Region')['Kum_antal_intensivvårdade'].max()
percentage_hospitalization_cases = total_number_of_hospitalized_county / total_number_of_cases_county * 100

data = {
    'County': percentage_hospitalization_cases.index,
    'Percentage Hospitalization': percentage_hospitalization_cases.values
}

fig = px.bar(
    data, 
    x='County', 
    y='Percentage Hospitalization', 
    labels={'Percentage Hospitalization': 'Percentage of Hospitalizations'},
    title='Percentage of Hospitalizations by County',
    color='Percentage Hospitalization'
).update_xaxes(categoryorder='total ascending')
fig.write_html("Visualizations/hospitalization_rate_county.html")
fig.show()

The lowest percentage of hospitallization we find in Västmanland. They are found in the bottom of vaccination rate for dose one and in the top we have Sörmland (or Södermanland), found in the middle. So it seems like it hard to prove a realation between dose number on a the minimazation of hospitalizations. But quite clear is that number of hospitalizations in Sörmland in relation to number of cases where a lot more then the rest of the country. 

#### 2.3 **Case Fatality Rate (CFR):** The percentage of confirmed COVID-19 cases that resulted in death. 

To answer this question we'll use the sheet with total number per county. Down below we can see the whole content in our sheet.

In [9]:
total_by_region = pd.read_excel("Data/Folkhalsomyndigheten_Covid19.xlsx", sheet_name="Totalt antal per region")
totals_per_region = pd.DataFrame(total_by_region)
totals_per_region.head(22)

,Region,Totalt_antal_fall,Fall_per_100000_inv,Totalt_antal_intensivvårdade,Totalt_antal_avlidna
0,Blekinge,30829,19371.449951,85,184
1,Dalarna,75091,26098.780273,260,544
2,Gotland,11874,19776.671875,46,82
3,Gävleborg,74803,26020.503418,332,754
4,Halland,108822,32349.047119,229,518
5,Jämtland Härjedalen,34347,26197.373535,95,197
6,Jönköping,89662,24583.161133,414,756
7,Kalmar,62810,25537.878418,135,385
8,Kronoberg,51460,25460.141602,162,410
9,Norrbotten,50755,20327.326904,267,459


If we divide the total number of deseaced with total number of cases (per county), we'll find the case fatality rate (CFR) per county. Below we find that the county with the highest CFR where Västernorrland. Quite surprisingly, the county was in the middle of our previous explorations. They where way below the highest number of cases as well as hospitalisations rate. Why more people died from the desease in Västernorrland is hard to tell from our data. One reason could be distance to emergency careand another coluld be higher mean age ([SCB](https://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__BE__BE0101__BE0101B/BefolkningMedelAlder/sortedtable/tableViewSorted/)). The conclusion to draw from below is that it were in Halland or Örebro (whom also had the highest number of citizens infected) fewer people died from the disease. As mentioned previous all conties had different approch to who they tested and not, hence the number of total cases might not be a valid data to use.

In [10]:
CFR = (totals_per_region['Totalt_antal_avlidna'] / totals_per_region['Totalt_antal_fall'])*100

data = {
    'County': totals_per_region['Region'],
    'Case Fatality Rate (CFR) %': CFR
}

fig = px.bar(
    data, 
    x='County', 
    y='Case Fatality Rate (CFR) %', 
    title='Case Fatality Rate by County',
    labels={'color': 'Percent'},
    color=CFR
)
fig.update_xaxes(categoryorder='total ascending')
fig.write_html("Visualizations/CFR_county.html")
fig.show()

If we instead look at the number of deceased and divide this with number of citizens, we can at least tell what county got strike the hardes during the pandemic considering deaths.

Below I'm using our data from 2.2 to aggregate the sum of the population in each county, and the data from above to aggregate the sum och total number of deceased by county. I calculate the death rate by dividing total deceased (county) by population (county).

In [11]:
# Data from 2.2 containing county, municipality, population a.s.o., and the data used for above graph.
population_by_county = vaccinated_municipallity_and_age.groupby('Län_namn')['Befolkning'].sum()
deceased_total_county = totals_per_region.groupby('Region')['Totalt_antal_avlidna'].sum()
death_rate_percent = (deceased_total_county.values / population_by_county.values) * 100

data={
    'County': totals_per_region['Region'],
    'Fatality Rate %': death_rate_percent
    
}
fig = px.bar(
    data,
    x='County', 
    y='Fatality Rate %',
    title='Fatality Rate by County',
    labels={'color': 'Percent'},
    color=death_rate_percent
)

fig.update_xaxes(categoryorder='total ascending')
fig.write_html("Visualizations/fatality_rate_county.html")
fig.show()

The above paint almost the same picture as previous graph, Västernorrland not only had the highest number of deaths if we compare it to number of cases but also the highest number of deaths compared to population. 